<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/VisitFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [4]:
visit = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Visit.csv')
visit.head()

<ipython-input-4-b128dc73cac8>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  visit = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Visit.csv')


,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,StartDTTM,EndDttm,Location,PricipalProblemCode,PricipalProblemName
0,19319,25325,หญิง,83,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
1,83224,25327,ชาย,64,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
2,160455,25336,ชาย,86,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
3,181216,25338,หญิง,82,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
4,568147,25357,ชาย,68,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN


In [5]:
visit.shape

(1340588, 11)

In [6]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

In [7]:
# Remove patient UIDs from visit['PatientUID'] that are not in the list
visit = visit[~visit['PatientUID'].isin(patient_uid_list)]
visit.shape

(1195332, 11)

In [22]:
visit.tail(10)

,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,StartDTTM,EndDttm,Location,PricipalProblemCode,PricipalProblemName
1340577,3170846,10575260,ชาย,73,Out Patient,ไทย,2566-04-10 13:54:10.467,NaN,โรคมะเร็ง (คลินิก)-D717,NaN,NaN
1340578,1622584,10575278,หญิง,56,Out Patient,ไทย,2566-04-10 14:01:04.763,NaN,ห้องจ่ายยากลาง-S1PHA,NaN,NaN
1340579,71920,10575283,ชาย,62,Out Patient,ไทย,2566-04-10 14:01:37.210,NaN,บูรณาการนอกเวลา (คลินิก)-OD191,NaN,NaN
1340580,972818,10575284,หญิง,73,Out Patient,ไทย,2566-04-10 14:01:43.087,NaN,PT (Physical therapy clinic)-D1201,20022000,Hemiparesis
1340581,2776128,10575301,ชาย,75,Out Patient,ลาว,2566-04-10 14:09:11.893,NaN,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,NaN,NaN
1340582,738719,10575302,ชาย,61,Out Patient,ไทย,2566-04-10 14:09:34.010,NaN,ห้องจ่ายยากลาง-S1PHA,NaN,NaN
1340583,2738720,10575306,หญิง,69,Out Patient,ไทย,2566-04-10 14:10:35.473,NaN,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN
1340584,2720813,10575337,ชาย,65,Out Patient,ไทย,2566-04-10 14:22:46.967,NaN,บูรณาการนอกเวลา (คลินิก)-OD191,NaN,NaN
1340586,135860,10575384,หญิง,80,Out Patient,ไทย,2566-04-10 14:45:26.267,NaN,PCU สามเหลี่ยม-DV263,Z743,Need for continuous supervision
1340587,2057262,10575392,ชาย,83,Out Patient,ไทย,2566-04-10 14:51:46.657,NaN,ห้องจ่ายยากลาง-S1PHA,NaN,NaN


In [29]:
selected = visit[['PatientUID', 'PatientvisitUID', 'Sex', 'Age', 'ENTY', 'NATNL', 'Location','PricipalProblemCode','PricipalProblemName']]
selected

,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,Location,PricipalProblemCode,PricipalProblemName
0,19319,25325,หญิง,83,Out Patient,ไทย,NaN,NaN,NaN
1,83224,25327,ชาย,64,Out Patient,ไทย,NaN,NaN,NaN
2,160455,25336,ชาย,86,Out Patient,ไทย,NaN,NaN,NaN
3,181216,25338,หญิง,82,Out Patient,ไทย,NaN,NaN,NaN
4,568147,25357,ชาย,68,Out Patient,ไทย,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1340582,738719,10575302,ชาย,61,Out Patient,ไทย,ห้องจ่ายยากลาง-S1PHA,NaN,NaN
1340583,2738720,10575306,หญิง,69,Out Patient,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN
1340584,2720813,10575337,ชาย,65,Out Patient,ไทย,บูรณาการนอกเวลา (คลินิก)-OD191,NaN,NaN
1340586,135860,10575384,หญิง,80,Out Patient,ไทย,PCU สามเหลี่ยม-DV263,Z743,Need for continuous supervision


In [38]:
import pandas as pd

# Assuming 'selected_columns' is the DataFrame containing the selected columns from 'visit'

# 1) Count PatientvisitUID for each PatientUID
count_patient_visit = selected.groupby('PatientUID')['PatientvisitUID'].count().rename('VisitCount')

# 2) Add new tables for Age_min and Age_max of each PatientUID
age_summary = selected.groupby('PatientUID')['Age'].agg(Age_min='min', Age_max='max')

# 3) Divide ENTY into 3 parts: Out Patient, InPatient, E. For PatientUID with null, fill using mode.
selected['ENTY'].fillna(selected['ENTY'].mode().iloc[0], inplace=True)

# Create new columns to count occurrences of 'Out Patient', 'InPatient', and 'E' for each 'PatientUID'
selected['Out_count'] = (selected['ENTY'] == 'Out Patient').astype(int)
selected['In_count'] = (selected['ENTY'] == 'InPatient').astype(int)
selected['Emergency'] = (selected['ENTY'] == 'E').astype(int)

# 4) Fill null values in the Location column using mode for each PatientUID
selected['Location'] = (selected.groupby('PatientUID')['Location'].agg(lambda x: x.mode().iloc[0]))

<ipython-input-38-f0d8781d1e50>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['ENTY'].fillna(selected['ENTY'].mode().iloc[0], inplace=True)
<ipython-input-38-f0d8781d1e50>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['Out_count'] = (selected['ENTY'] == 'Out Patient').astype(int)


In [40]:
import pandas as pd
import statistics

# Assuming 'selected' is a valid DataFrame with columns 'PatientUID' and 'PricipalProblemName'

def mode_with_default(x):
    try:
        return x.mode().iloc[0]
    except IndexError:
        return None

selected['PricipalProblemName'] = selected.groupby('PatientUID')['PricipalProblemName'].agg(mode_with_default)
selected['PricipalProblemCode'] = selected.groupby('PatientUID')['PricipalProblemCode'].agg(mode_with_default)


In [50]:
visit['PricipalProblemName'].nunique()

13808

In [51]:
visit['PricipalProblemCode'].nunique()

14645

In [56]:
visit[(visit['PricipalProblemCode'].notnull()) & (visit['PricipalProblemName'].isnull())]

,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,StartDTTM,EndDttm,Location,PricipalProblemCode,PricipalProblemName


In [44]:
selected

,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,Location,PricipalProblemCode,PricipalProblemName,Out_count,In_count,Emergency
0,19319,25325,หญิง,83,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
1,83224,25327,ชาย,64,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
2,160455,25336,ชาย,86,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
3,181216,25338,หญิง,82,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
4,568147,25357,ชาย,68,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1340582,738719,10575302,ชาย,61,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
1340583,2738720,10575306,หญิง,69,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
1340584,2720813,10575337,ชาย,65,Out Patient,ไทย,NaN,NaN,NaN,1,0,0
1340586,135860,10575384,หญิง,80,Out Patient,ไทย,NaN,NaN,NaN,1,0,0


In [ ]:
# 5) Group by 'PatientUID' as index, including all the calculated columns
grouped_table = selected.groupby('PatientUID').agg({
    'PatientvisitUID': 'count',   # 1) Count PatientvisitUID
    'Age': ['min', 'max'],        # 2) Age_min and Age_max
    'Out_count': 'sum',         # 3) Count Out Patient occurrences for each PatientUID
    'In_count': 'sum',           # 3) Count InPatient occurrences for each PatientUID
    'Emergency': 'sum',           # 3) Count E occurrences for each PatientUID
    'Location': 'first',          # 4) Location with null filled using mode
    'Sex': 'first',               # Add Sex (use the first value as all should be the same)
    'NATNL': 'first',             # Add NATNL (use the first value as all should be the same)
})

# Flatten the multi-index columns
grouped_table.columns = ['VisitCount', 'AgeMin', 'AgeMax', 'OutCount', 'InCount', 'EmergencyCount', 'LocationMode', 'PricipalProblemCodeMode','PricipalProblemNameMode','Sex', 'NATNL']

# Show the result
grouped_table